## Overview

The goal of this project is to Spending based on the other available customer information. This file contains data about whether or not different consumers made a purchase in response to a test mailing of a certain catalog and, in case of a purchase, how much money each consumer spent.

We will be using 9 different classification algorithms shown below - 

1. K-Nearest Neighbors
2. Decision Trees
3. Naive Bayes
4. Linear Regression
5. Support Vector Machines
6. Elastic Net Regression
7. Random Forests
8. Gradient Boosting
9. XGBoost

We will also try this in two different parts - 
1. Part A - Without using the information in the purchase indicator
2. Part B - Using the information in the purchase indicator

### Important Packages

We will be using the scikit-learn package in python to train models for our prediction problem. We will also use matplotlib to visualize our results.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import neighbors
from sklearn import tree
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
## Cleaning the data

data = pd.read_csv('HW3.csv')

data = data.drop(labels = 'sequence_number', axis = 1)
data.head()

,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,source_r,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,0,0,1,0,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,127.87
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0.00
2,1,0,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127.48
3,1,0,1,0,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0.00
4,1,0,1,0,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0.00


### Part A - Split into train, test and validation

To avoid overfitting, we will split our data into 2 parts - train and test.


We will use the validation set to select the best model from all the different hyperparameters and then finally compare the results of all the different models on the testing dataset to decide which model works the best.

The split that I have chosen - 

- Train = 70% 
- Test = 30%

We will first attempt to target part 1 of this exercise - without using the purchase indicator

In [3]:
## Splitting the dataset for two parts of the question

data_part2 = data[data['Purchase']==1]

## Training and testing 
X = data.drop(labels = ['Purchase','Spending'], axis = 1)
y = data['Spending']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)


## Model Selection

We will use nested cross-validation to select the best model out of the all choices we have.

We follow the steps below for each model - 

1. We select the list of parameters we want to optimize over for each model and put it into a dictionary 
2. Set up the inner cross-validation object by using this parameter grid
3. Set up the outer cross-validation object.
5. Create a grid search object using the inner cv object. This will be used to find the best parameter for each outer fold.
4. Get the cross validation score using the grid search object as the estimator and the outer_cv as the cross validation parameter


In [4]:
## Setting hyperparameters

### Create model objects

knn = make_pipeline(MinMaxScaler(), neighbors.KNeighborsRegressor())
dt = tree.DecisionTreeRegressor()
lm = make_pipeline(StandardScaler(), LinearRegression())
svm_regressor = make_pipeline(MinMaxScaler(), SVR())
gm_regressor = make_pipeline(MinMaxScaler(), GradientBoostingRegressor())
enet_regressor = make_pipeline(MinMaxScaler(), ElasticNet())
rf_regressor = make_pipeline(MinMaxScaler(), RandomForestRegressor())
xgb_regressor = make_pipeline(MinMaxScaler(), XGBRegressor(verbosity = 0))

###### Create Parameter List

## KNN
k_range = list(range(1,10))
knn_params = dict(kneighborsregressor__n_neighbors = k_range, 
                  kneighborsregressor__weights = ['uniform','distance'], 
                  kneighborsregressor__p = [1,2,3])

## Decision Tree
depth_range = list(range(1,10))
min_samples_range = list(range(2,10))
impurity_decrease_range = list(np.linspace(0.1,0.5,5))

dt_params = dict(criterion = ['squared_error','absolute_error'], 
                 splitter = ['best','random'],
                 max_depth = depth_range, 
                 max_features = ['auto','sqrt','log2',None], 
                 random_state = [456],
                 min_impurity_decrease = impurity_decrease_range)

## Linear Regression and Elastic Net


l1r_range = [0,0.5,1]

enet_params = dict(elasticnet__alpha = [0.1,1,10],
                   elasticnet__l1_ratio = l1r_range)

## Support Vector Machine

c_range = [0.1, 1, 10]

svm_params = {"svr__C": c_range,
              "svr__degree": list(range(1,5)),
              "svr__kernel": ['rbf'],
              "svr__max_iter": [10000]
            }

## Random Forest
rf_params = {
    "randomforestregressor__n_estimators": [100,200,500],
    "randomforestregressor__max_depth": list(range(5,10)),
    "randomforestregressor__bootstrap": [True,False]}


## Gradient Boosting

gb_params = {
    "gradientboostingregressor__n_estimators": [100,500,1000,2000],
    "gradientboostingregressor__learning_rate": [0.001,0.01,0.1],
    "gradientboostingregressor__loss": ['squared_error']}

## XGBoost

xgb_params = {
    "xgbregressor__n_estimators": [100,200,500],
    "xgbregressor__max_depth": depth_range,
    "xgbregressor__learning_rate": [0.01,0.1]}

## Neural Networks
## https://stackoverflow.com/questions/44132652/keras-how-to-perform-a-prediction-using-kerasregressor
def create_model(activation='relu', nb_hidden=10, nb_hidden_2 = 10, nb_hidden_3 = 10):
    model = Sequential()
    model.add(Dense(nb_hidden, input_dim=22, activation=activation))
    model.add(Dense(nb_hidden_2, activation=activation))
    model.add(Dense(nb_hidden_3, activation=activation))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


activations = ['relu', 'tanh', 'sigmoid']
nb_hiddens = np.array([100, 200])
nb_hiddens_2 = np.array([10,20])
nb_hiddens_3 = np.array([5,10])

param_grid = dict(kerasregressor__activation=activations,
                  kerasregressor__nb_hidden=nb_hiddens, 
                  kerasregressor__nb_hidden_2 = nb_hiddens_2, 
                  kerasregressor__nb_hidden_3 = nb_hiddens_3)

model = make_pipeline(StandardScaler(), 
                      KerasRegressor(build_fn=create_model,
                                     epochs=2,
                                     batch_size=256,
                                     verbose=0))




In [5]:
##### Set up inner and outerCV loops

inner_cv = KFold(n_splits = 5, shuffle = True, random_state = 456)
outer_cv = KFold(n_splits = 5, shuffle = True, random_state = 456)

### Use metric

scoring = 'neg_mean_squared_error'


### Create Grid Search estimators

knn_gs = GridSearchCV(estimator = knn, param_grid = knn_params, 
                      scoring = scoring, cv = inner_cv)
dt_gs = GridSearchCV(estimator = dt, param_grid = dt_params, 
                     scoring = scoring, cv = inner_cv)
svm_gs = GridSearchCV(estimator = svm_regressor, param_grid = svm_params, 
                      scoring = scoring, cv = inner_cv)
enet_gs = GridSearchCV(estimator = enet_regressor, param_grid = enet_params, 
                      scoring = scoring, cv = inner_cv)
rf_gs = GridSearchCV(estimator = rf_regressor, param_grid = rf_params, 
                    scoring = scoring, cv = inner_cv)
gbm_gs = GridSearchCV(estimator = gm_regressor, param_grid = gb_params, 
                      scoring = scoring, cv = inner_cv)
xgb_gs = GridSearchCV(estimator = xgb_regressor, param_grid = xgb_params, 
                      scoring = scoring, cv = inner_cv)
nn_gs = GridSearchCV(estimator=model, param_grid=param_grid,
                  cv= inner_cv, scoring=scoring)

### performing Nested cross validation using the inner and outer loops
dt_score = cross_val_score(estimator = dt_gs, X = X_train, y = y_train, 
                           cv = outer_cv, scoring = scoring)
knn_score = cross_val_score(estimator = knn_gs, X = X_train, y = y_train, 
                            cv = outer_cv, scoring = scoring)
enet_score = cross_val_score(estimator = enet_gs, X = X_train, y = y_train, 
                              cv = outer_cv, scoring = scoring)
svm_score = cross_val_score(estimator = svm_gs, X = X_train, y = y_train, 
                            cv = outer_cv, scoring = scoring)

lm_score = cross_val_score(estimator = lm, X = X_train, y = y_train, cv = outer_cv,
                           scoring = scoring)
rf_score = cross_val_score(estimator = rf_gs, X = X_train, y = y_train, cv = outer_cv,
                           scoring = scoring)
gbm_score = cross_val_score(estimator = gbm_gs, X = X_train, y = y_train, cv = outer_cv,
                           scoring = scoring)
xgb_score = cross_val_score(estimator = xgb_gs, X = X_train, y = y_train, cv = outer_cv,
                           scoring = scoring)
nn_score = cross_val_score(estimator = nn_gs, X = X_train, y = y_train, cv = outer_cv,
                           scoring = scoring)

### Part A - Results using mean squared error

It looks like the xgboost algorithm does the best on our training data with an mean squared error of 15137.43. We will use this model to fit our data. The model performances are listed below - 

1. knn = 29025.11 ± 3949.38
2. dt = 18257.52 ± 4874.32
3. lm = 15919.46 ± 3457.37
4. enet = 32206.54 ± 3480.73
5. svm = 32206.54 ± 4180.3
6. random forest = 16015.65 ± 3704.79
7. gradient boost = 15597.9 ± 3865.95
8. xgboost = 15137.43 ± 3762.25
9. nn = 45442.36 ± 4196.29

In [7]:
print("knn = " + str(np.round(-knn_score.mean(),2)) + " " + u"\u00B1" + " " 
      + str(np.round(knn_score.std(),2)))
print("dt = " + str(np.round(-dt_score.mean(),2)) + " " + u"\u00B1" + " " 
      + str(np.round(dt_score.std(),2)))
print("lm = " + str(np.round(-lm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(lm_score.std(),2)))
print("enet = " + str(np.round(-svm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(enet_score.std(),2)))
print("svm = " + str(np.round(-svm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(svm_score.std(),2)))
print("random forest = " + str(np.round(-rf_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(rf_score.std(),2)))
print("gradient boost = " + str(np.round(-gbm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(gbm_score.std(),2)))
print("xgboost = " + str(np.round(-xgb_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(xgb_score.std(),2)))
print("nn = " + str(np.round(-nn_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(nn_score.std(),2)))


knn = 29025.11 ± 3949.38
dt = 18257.52 ± 4874.32
lm = 15919.46 ± 3457.37
enet = 32206.54 ± 3480.73
svm = 32206.54 ± 4180.3
random forest = 16015.65 ± 3704.79
gradient boost = 15597.9 ± 3865.95
xgboost = 15137.43 ± 3762.25
nn = 45442.36 ± 4196.29


## Re-training and evaluation

Now we re-train our XGBoost model on the entire training dataset and evaluate it on the testing set. It looks like our model performs well on the testing set as well and gives us a mean squared error of 11236.


In [8]:
## Since Xgboost performs the best, we will use that for our models ahead
X = data.drop(labels = ['Purchase','Spending'], axis = 1)
y = data['Spending']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

final_model = xgb_gs.fit(X_train,y_train)

y_pred = final_model.predict(X_test)
mean_squared_error(y_test, y_pred)

11236.089168417564

## Part B - Using purchase indicator

Using the information in the purchase indicator, we repeat the steps of the Nested Cross Validation, but this time only with the data with purchases.

In [9]:
## Training and testing 
X_1 = data_part2.drop(labels = ['Spending'], axis = 1)
y_1 = data_part2['Spending']

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, train_size = 0.8)

##### Set up inner and outerCV loops

inner_cv = KFold(n_splits = 5, shuffle = True, random_state = 456)
outer_cv = KFold(n_splits = 5, shuffle = True, random_state = 456)

### Use metric

scoring = 'neg_mean_squared_error'


### Create Grid Search estimators

knn_gs = GridSearchCV(estimator = knn, param_grid = knn_params, 
                      scoring = scoring, cv = inner_cv)
dt_gs = GridSearchCV(estimator = dt, param_grid = dt_params, 
                     scoring = scoring, cv = inner_cv)
svm_gs = GridSearchCV(estimator = svm_regressor, param_grid = svm_params, 
                      scoring = scoring, cv = inner_cv)
enet_gs = GridSearchCV(estimator = enet_regressor, param_grid = enet_params, 
                      scoring = scoring, cv = inner_cv)
rf_gs = GridSearchCV(estimator = rf_regressor, param_grid = rf_params, 
                    scoring = scoring, cv = inner_cv)
gbm_gs = GridSearchCV(estimator = gm_regressor, param_grid = gb_params, 
                      scoring = scoring, cv = inner_cv)
xgb_gs = GridSearchCV(estimator = xgb_regressor, param_grid = xgb_params, 
                      scoring = scoring, cv = inner_cv)
nn_gs = GridSearchCV(estimator=model, param_grid=param_grid,
                  cv= inner_cv, scoring=scoring)

### performing Nested cross validation using the inner and outer loops
dt_score = cross_val_score(estimator = dt_gs, X = X_train_1, y = y_train_1, 
                           cv = outer_cv, scoring = scoring)
knn_score = cross_val_score(estimator = knn_gs, X = X_train_1, y = y_train_1, 
                            cv = outer_cv, scoring = scoring)
enet_score = cross_val_score(estimator = enet_gs, X = X_train_1, y = y_train_1, 
                              cv = outer_cv, scoring = scoring)
svm_score = cross_val_score(estimator = svm_gs, X = X_train_1, y = y_train_1, 
                            cv = outer_cv, scoring = scoring)

lm_score = cross_val_score(estimator = lm, X = X_train_1, y = y_train_1, cv = outer_cv,
                           scoring = scoring)
rf_score = cross_val_score(estimator = rf_gs, X = X_train_1, y = y_train_1, cv = outer_cv,
                           scoring = scoring)
gbm_score = cross_val_score(estimator = gbm_gs, X = X_train_1, y = y_train_1, cv = outer_cv,
                           scoring = scoring)
xgb_score = cross_val_score(estimator = xgb_gs, X = X_train_1, y = y_train_1, cv = outer_cv,
                           scoring = scoring)
nn_score = cross_val_score(estimator = nn_gs, X = X_train_1, y = y_train_1, cv = outer_cv,
                           scoring = scoring)

### Part A - Results using mean squared error

It looks like the xgboost algorithm does the best on our training data with an mean squared error of 26259. We will use this model to fit our data. The model performances are listed below - 

1. knn = 42827.45 ± 11978.29
2. dt = 30231.82 ± 7507.77
3. lm = 26474.49 ± 7750.66
4. enet = 47477.56 ± 7746.15
5. svm = 47477.56 ± 12520.09
6. random forest = 25682.17 ± 7412.36
7. gradient boost = 25156.34 ± 7755.74
8. xgboost = 26259.16 ± 7251.22

In [10]:
print("knn = " + str(np.round(-knn_score.mean(),2)) + " " + u"\u00B1" + " " 
      + str(np.round(knn_score.std(),2)))
print("dt = " + str(np.round(-dt_score.mean(),2)) + " " + u"\u00B1" + " " 
      + str(np.round(dt_score.std(),2)))
print("lm = " + str(np.round(-lm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(lm_score.std(),2)))
print("enet = " + str(np.round(-svm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(enet_score.std(),2)))
print("svm = " + str(np.round(-svm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(svm_score.std(),2)))
print("random forest = " + str(np.round(-rf_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(rf_score.std(),2)))
print("gradient boost = " + str(np.round(-gbm_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(gbm_score.std(),2)))
print("xgboost = " + str(np.round(-xgb_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(xgb_score.std(),2)))
print("nn = " + str(np.round(-nn_score.mean(),2)) + " " + u"\u00B1" + " "
      + str(np.round(nn_score.std(),2)))

knn = 42827.45 ± 11978.29
dt = 30231.82 ± 7507.77
lm = 26474.49 ± 7750.66
enet = 47477.56 ± 7746.15
svm = 47477.56 ± 12520.09
random forest = 25682.17 ± 7412.36
gradient boost = 25156.34 ± 7755.74
xgboost = 26259.16 ± 7251.22
nn = nan ± nan


## Comparing the results of Part A and B

1. Overall - it seems like models in part A do better than models in part B
2. If we look at observations that have purchase = 1, and then calculate the mse for only those observations, we see that models in part B do better

One reason for this is in Part A, models learn from zero spend observations and tend to underpredict on the purchase = 1 observations. In Part B, this does not happen as there are no zero spend observations which skew the fit of the model.

In [11]:
final_model_1 = xgb_gs.fit(X_train_1,y_train_1)

y_pred_1 = final_model_1.predict(X_test_1)
mean_squared_error(y_test_1, y_pred_1)

31501.254804623717